In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class SmallMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(10, 20, bias=True)
        self.bn1 = nn.BatchNorm1d(20)
        self.fc2 = nn.Linear(20, 20, bias=True)
        self.bn2 = nn.BatchNorm1d(20)
        self.fc3 = nn.Linear(20, 2)

    def forward(self, x):
        x = self.fc1(x)
        x = self.bn1(x)
        x = torch.tanh(x)
        x = self.fc2(x)
        x = self.bn2(x)
        x = torch.tanh(x)
        x = self.fc3(x)
        return x

model = SmallMLP()
optimizer = torch.optim.SGD(model.parameters(), lr=0.1)

for _ in range(100):
    X = torch.randn(32, 10)
    Y = torch.randint(0, 2, (32,))

    logits = model(X)
    loss = F.cross_entropy(logits, Y)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

model.eval()

with torch.no_grad():
    W = model.fc1.weight
    b = model.fc1.bias

    gamma = model.bn1.weight
    beta = model.bn1.bias
    mean = model.bn1.running_mean
    var = model.bn1.running_var
    epsilon = model.bn1.eps

    scale = gamma / torch.sqrt(var + epsilon)

    W_folded = W * scale.view(-1, 1)
    b_folded = beta + scale * (b - mean)

    fc1_folded = nn.Linear(10, 20)
    fc1_folded.weight.copy_(W_folded)
    fc1_folded.bias.copy_(b_folded)

x_input = torch.randn(5, 10)

with torch.no_grad():
    out_original = model.bn1(model.fc1(x_input))
    out_folded = fc1_folded(x_input)

diff = (out_original - out_folded).abs().max()

print(f"Max Difference: {diff.item():.8f}")

Max Difference: 0.00000036
